<a href="https://colab.research.google.com/github/chugmn/NYP-PUBLIC/blob/main/Bank_Authentication_Notes_Version_1_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt"
columns = ["variance", "skewness", "kurtosis", "entropy", "class"]
data = pd.read_csv(url, header=None, names=columns)

# Display the first few rows
print(data.head())


   variance  skewness  kurtosis  entropy  class
0   3.62160    8.6661   -2.8073 -0.44699      0
1   4.54590    8.1674   -2.4586 -1.46210      0
2   3.86600   -2.6383    1.9242  0.10645      0
3   3.45660    9.5228   -4.0112 -3.59440      0
4   0.32924   -4.4552    4.5718 -0.98880      0


In [ ]:
# Separate features and target
X = data.drop("class", axis=1).values
y = data["class"].values

# Manually split data into 80% train and 20% test
split_index = int(0.8 * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Normalize the features
X_train = (X_train - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)
X_test = (X_test - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)  # Use training mean/std



In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Build the model
#model = keras.Sequential([
#    layers.Input(shape=(X_train.shape[1],)),  # Use Input layer for input shape
#    layers.Dense(16, activation='relu'),
#    layers.Dense(8, activation='relu'),
#    layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
#])

# Compile the model
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 2nd try :
#def build_model():
 #   model = tf.keras.Sequential([
#      tf.keras.layers.Input(shape=(X.shape[1],)),  # Specify input shape here
#        tf.keras.layers.Dense(16, activation='relu'),
#        tf.keras.layers.Dense(8, activation='relu'),
#        tf.keras.layers.Dense(1, activation='sigmoid')
#    ])
#    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#    return model

# 3rd try :

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential, Model

def build_model():
    inputs = Input(shape=(X.shape[1],))  # Explicitly define input
    x = Dense(16, activation='relu')(inputs)
    x = Dense(8, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)  # Wrap in a functional model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.2)


Epoch 1/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7894 - loss: 1.1414 - val_accuracy: 0.9455 - val_loss: 0.2937
Epoch 2/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9827 - loss: 0.0550 - val_accuracy: 0.9636 - val_loss: 0.1878
Epoch 3/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9999 - loss: 0.0104 - val_accuracy: 0.9591 - val_loss: 0.1694
Epoch 4/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9920 - loss: 0.0196 - val_accuracy: 0.9500 - val_loss: 0.2104
Epoch 5/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0029 - val_accuracy: 0.9591 - val_loss: 0.1828
Epoch 6/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0026 - val_accuracy: 0.9591 - val_loss: 0.1825
Epoch 7/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 0.9591 - val_loss: 0.1809
Epoch 8/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0014 - val_accuracy: 0.9591 - v

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

# Assuming X and y are your input and target data respectively
# Set number of splits (folds) for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# To store the accuracy of each fold
cv_scores = []

# Define the model architecture in a function for reusability
def create_model():
    model = Sequential([
        Dense(12, input_dim=X.shape[1], activation='relu'),
        Dense(8, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.01),
                  loss=BinaryCrossentropy(),
                  metrics=['accuracy'])
    return model

# Cross-validation loop
for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Create a new model instance for each fold
    model = create_model()

    # Train the model
    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=50, batch_size=10, verbose=0)

    # Evaluate the model on the validation set
    scores = model.evaluate(X_val, y_val, verbose=0)
    cv_scores.append(scores[1])  # scores[1] is accuracy

# Print cross-validation results
print(f"Cross-Validation Accuracies: {cv_scores}")
print(f"Mean Accuracy: {np.mean(cv_scores)}")
print(f"Standard Deviation: {np.std(cv_scores)}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Cross-Validation Accuracies: [1.0, 1.0, 1.0, 1.0, 1.0]
Mean Accuracy: 1.0
Standard Deviation: 0.0
